In [5]:
import pandas as pd
import dask
import lightgbm as lgb
import dask.dataframe as dd
from dask.diagnostics.progress import ProgressBar
from functools import reduce
import gc
dask.set_options(temporary_directory='/root/mount/tmp')

In [ ]:
train_count_accum = pd.read_feather('train_count_accum.ftr')
train_origin_uniq_nextClick = pd.read_feather('train_origin_uniq_nextClick.ftr')

In [ ]:
sd1 = dd.from_pandas(train_count_accum, npartitions=3)
sd2 = dd.from_pandas(train_origin_uniq_nextClick, npartitions=3)

In [ ]:
sd1.to_hdf('train_count_accum.h5', 'train_count_accum')
sd2.to_hdf('train_origin_uniq_nextClick.h5', 'train_origin_uniq_nextClick')

In [ ]:
sd1 = dd.read_hdf('train_count_accum.h5', 'train_count_accum')
sd2 = dd.read_hdf('train_origin_uniq_nextClick.h5', 'train_origin_uniq_nextClick')

In [ ]:
sd3 = sd1.merge(sd2, on=['index'])

In [ ]:
with ProgressBar():
    sd3.to_hdf('train_features.h5', 'train_features')

In [2]:
cols = [
    'ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'is_attributed', 
    'app_count', 'ip_count', 'app_channel_count',
    'ip_device_count', 'ip_day_count', 'app_channel_hour_count',
    'app_channel_day_count', 'app_channel_day_hour_count', 
    'ip_countAccum', 'channel_countAccum', 'app_countAccum', 
    'device_countAccum', 'app_channel_countAccum', 'app_channel_day_countAccum',
    'channel_day_hour_countAccum', 'device_channel_day_hour_countAccum',
    'app_channel_day_hour_countAccum', 'app_device_channel_day_hour_countAccum', 
    'ip_day_countAccum', 'ip_device_countAccum', 
    'app_uniq_ip_countUniq', 'app_day_uniq_ip_countUniq', 'app_device_channel_uniq_ip_countUniq',
    'app_hour_channel_uniq_ip_countUniq', 'ip_uniq_channel_countUniq',
    'ip_uniq_app_countUniq', 'ip_uniq_hour_countUniq',
    'ip_uniq_os_countUniq', 'app_channel_hour_uniq_os_countUniq',
    'app_channel_day_hour_uniq_os_countUniq', 'ip_nextClick',
    'channel_nextClick', 'ip_device_nextClick', 'channel_day_nextClick',
    'app_channel_nextClick', 'ip_app_nextClick', 'ip_app_os_nextClick',
    'ip_app_os_device_nextClick', 'ip_app_os_device_channel_nextClick'
]

In [ ]:
train = pd.read_hdf('train_features.h5', 'train_features', stop=100000000, columns=cols)

In [2]:
train = pd.read_feather('data/train_features.ftr')
valid = pd.read_feather('data/valid_features.ftr')

In [3]:
all_features = pd.concat([train, valid], ignore_index=True)

In [7]:
all_features.shape

(184903890, 30)

In [6]:
del train
del valid
gc.collect()

2461

In [9]:
features = [x for x in all_features.columns if x not in ['index', 'click_time', 'is_attributed']]
categorical = ['ip', 'app', 'device', 'os', 'channel', 'hour']
respond = 'is_attributed'

print('converting...')
xgtrain = lgb.Dataset(all_features[features].values, 
                      label=all_features[respond].values,
                      feature_name=features,
                      categorical_feature=categorical
                     )

print('save binary')
xgtrain.save_binary('data/all_features.bin')

converting...
save binary


In [10]:
xgtrain.save_binary('data/all_features.bin')